## Sparsity experiment

In [1]:
import torch
import numpy as np

from datasets.dataset import transform_dataset, kfold_dataset
from R2Ntab import train as train_r2ntab, R2Ntab
from DRNet import train as train_drnet, DRNet

import sys

import matplotlib.pyplot as plt
import numpy as np

In [2]:
DRNetAccus = []
DRNetAccusOverTime = []
DRNetRuleConditions = []
R2NTabAccus = []
R2NTabAccusOverTime = []
R2NTabRuleConditions = []
runs = 10
# Read datasets
for name in ['adult', 'heloc', 'house', 'magic']:
    print('dataset:', name)
    for run in range(runs):
        print('  run:', run+1)
        X, Y, X_headers, Y_headers = transform_dataset(name, method='onehot-compare', negations=False, labels='binary')
        datasets = kfold_dataset(X, Y, shuffle=1)
        X_train, X_test, Y_train, Y_test = datasets[0]
        train_set = torch.utils.data.TensorDataset(torch.Tensor(X_train.to_numpy()), torch.Tensor(Y_train))
        test_set = torch.utils.data.TensorDataset(torch.Tensor(X_test.to_numpy()), torch.Tensor(Y_test))
        
        # Train DR-Net
        # Default learning rate (1e-2), and_lam (1e-2), and and_lam (1e-5) usually work the best. A large epochs number is necessary for a sparse rule set i.e 10000 epochs.
        net = DRNet(train_set[:][0].size(1), 50, 1)
        acc = train_drnet(net, train_set, test_set=test_set, device='cpu', lr_rules=1e-2, epochs=1000, batch_size=400,
                    and_lam=1e-2, or_lam=1e-5, num_alter=500)
        DRNetAccusOverTime.append(acc)
        
        rule_set = net.get_rules(X_headers)
        DRNetAccus.append((net.predict(np.array(X_test)) == Y_test).mean())
        DRNetRuleConditions.append(sum(map(len, rule_set)))
        
        # Train R2N-tab
        # Default learning rate (1e-2), and_lam (1e-2), and and_lam (1e-5) usually work the best. 
        net = R2Ntab(train_set[:][0].size(1), 50, 1)
        acc = train_r2ntab(net, train_set, test_set=test_set, device='cpu', lr_cancel=5e-3, lr_rules=1e-2, epochs=1000, batch_size=400,
                     and_lam=1e-2, or_lam=1e-5, cancel_lam=1e-4, num_alter=500)
        R2NTabAccusOverTime.append(acc)
        
        rule_set = net.get_rules(X_headers)
        R2NTabAccus.append((net.predict(np.array(X_test)) == Y_test).mean())
        R2NTabRuleConditions.append(sum(map(len, rule_set)))

import json

with open(f'{DRNetAccus}.txt', 'w') as filehandle:
    json.dump(DRNetAccus, filehandle)

with open(f'{DRNetAccusOverTime }.txt', 'w') as filehandle:
    json.dump(DRNetAccusOverTime , filehandle)

with open(f'{DRNetRuleConditions}.txt', 'w') as filehandle:
    json.dump(DRNetRuleConditions, filehandle)

with open(f'{R2NTabAccus}.txt', 'w') as filehandle:
    json.dump(R2NTabAccus, filehandle)

with open(f'{R2NTabAccusOverTime}.txt', 'w') as filehandle:
    json.dump(R2NTabAccusOverTime, filehandle)

with open(f'{R2NTabRuleConditions}.txt', 'w') as filehandle:
    json.dump(R2NTabRuleConditions, filehandle)

dataset: adult
  run: 1


Epoch: 100%|██████████| 1000/1000 [03:36<00:00,  4.63it/s, loss=0.546, epoch acc
2023-06-09 01:37:36.303156: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-09 01:37:37.058204: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-09 01:37:37.062305: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-09 01:37:38.752579: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Epoch: 100%|██████████| 1000/1000 [04:06<00:00,  4.05it/s, rules cancelled=50, l


  run: 2


Epoch: 100%|██████████| 1000/1000 [03:33<00:00,  4.69it/s, loss=0.551, epoch acc
Epoch: 100%|██████████| 1000/1000 [04:08<00:00,  4.03it/s, rules cancelled=55, l


  run: 3


...

  run: 9


Epoch: 100%|██████████| 1000/1000 [02:12<00:00,  7.54it/s, loss=0.495, epoch acc
Epoch: 100%|██████████| 1000/1000 [02:31<00:00,  6.59it/s, rules cancelled=20, l


  run: 10


Epoch: 100%|██████████| 1000/1000 [02:12<00:00,  7.52it/s, loss=0.49, epoch accu
Epoch: 100%|██████████| 1000/1000 [02:31<00:00,  6.60it/s, rules cancelled=25, l


OSError: [Errno 36] File name too long: '[0.8309298856290402, 0.820321564727333, 0.8291065804740594, 0.8319244157135753, 0.840046411403945, 0.8334162108403779, 0.8286093154317918, 0.8307641306149511, 0.8345764959390022, 0.8334162108403779, 0.7184512428298279, 0.7174952198852772, 0.7151051625239006, 0.7088910133843213, 0.7055449330783938, 0.7103250478011472, 0.7131931166347992, 0.7165391969407265, 0.7021988527724665, 0.7079349904397706, 0.8556067588325653, 0.8624094799210007, 0.8613122668422207, 0.8626289225367566, 0.8608733816107088, 0.8635066929997806, 0.8648233486943164, 0.8648233486943164, 0.8562650866798333, 0.8659205617730963, 0.8443743427970557, 0.8509463722397477, 0.8477917981072555, 0.8454258675078864, 0.8422712933753943, 0.8406940063091483, 0.8446372239747634, 0.8464773922187171, 0.8446372239747634, 0.8485804416403786].txt'

# Rule set sparsity experiment
## DRNet vs R2NTab

In [ ]:
labels = ['adult', 'heloc', 'house', 'magic']
legend_labels = ['DRNet', 'R2N-Tab']

x_positions = np.arange(len(labels))

bar_width = 0.35

DRNetConditions = []
R2NTabConditions = []
for i in range(4):
    start = i*runs
    end = (i+1)*runs
    DRNetConditionsPerDataset = sum(DRNetRuleConditions[start:end])/len(DRNetRuleConditions[start:end])
    R2NTabConditionsPerDataset = sum(R2NTabRuleConditions[start:end])/len(R2NTabRuleConditions[start:end])
    DRNetConditions.append(DRNetConditionsPerDataset)
    R2NTabConditions.append(R2NTabConditions)

DRNetRuleConditions = np.array(DRNetConditions)
R2NTabRuleConditions = np.array(R2NTabConditions)

data = np.stack((DRNetRuleConditions, R2NTabRuleConditions), axis=1)

fig, ax = plt.subplots()
for i in range(data.shape[1]):
    ax.bar(x_positions + i * bar_width, data[:, i], bar_width, label=legend_labels[i])

ax.set_xlabel('Dataset')
ax.set_ylabel('# Conditions')
ax.set_title('# conditions in rule set per method per dataset for 1000 epochs')

ax.set_xticks(x_positions + bar_width / 2)
ax.set_xticklabels(labels)

ax.legend()

plt.savefig('sparsityexperiment1-1e-4.png')

plt.show()

# Performance per method
## DRNet vs R2NTab

In [ ]:
labels = ['adult', 'heloc', 'house', 'magic']
for index, dataset in enumerate(labels):
    start = index*runs
    end = (index+1)*runs
    dataDRNet = DRNetAccusOverTime[start:end]
    dataDRNet = np.array(dataDRNet)
    dataDRNet = np.mean(dataDRNet, axis=0)
    
    dataR2NTab = R2NTabAccusOverTime[start:end]
    dataR2NTab = np.array(dataR2NTab)
    dataR2NTab = np.mean(dataR2NTab, axis=0)
    plt.plot(dataDRNet)
    plt.plot(dataR2NTab)
    plt.savefig(f'accuracyexperiment1-1e-4.png')
    plt.show()